# Vertex Tabular Binary Classification with .CustomJob() / Container

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/jchavezar/vertex-ai-samples/blob/main/vertex-custom-ml/sklearn/container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
</table>

**Use case: predict if a customer will buy on return visit.**

The ecommerce dataset has different training features:
- latest_ecommerce_progress
- bounces
- time_on_site
- pageviews
- source
- medium
- channel_grouping
- device_category
- country

The label: will_buy_on_return_visit

Data is imbalanced

## Colab Only

*Uncomment and execute if colab*

In [1]:
#! pip3 install --upgrade google-cloud-aiplatform -q google-cloud-bigquery db-dtypes

In [2]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

In [3]:
# from google.colab import auth
# auth.authenticate_user()

## Import Libraries

In [4]:
from google.cloud import aiplatform as aip

## Variables

In [5]:
IMAGE_TRAIN_URI = "gcr.io/jchavezar-demo/sklearn-train"
IMAGE_PREDICTION_URI = "gcr.io/jchavezar-demo/ecommerce:fast-onnx"
MODEL_URI = "gs://vtx-models/ecommerce/sklearn"

## Initialize Vertex Project

*Remember to change project and staging_bucket*

In [6]:
aip.init(project='jchavezar-demo', staging_bucket='gs://vtx-staging')


## Train your Model using Vertex (Containers)

In [7]:
worker_pool_specs=[
    {
        "machine_spec": {
            "machine_type": "n1-standard-4"
        },
        "replica_count" : 1,
        "container_spec": {
            "image_uri": IMAGE_TRAIN_URI
        }
    }
]

my_job = aip.CustomJob(
    display_name = "sklearn-customjob-train",
    worker_pool_specs = worker_pool_specs,
    base_output_dir = MODEL_URI,
)

my_job.run()

/home/sockcop/.local/lib/python3.9/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Creating CustomJob
CustomJob created. Resource name: projects/569083142710/locations/us-central1/customJobs/8897818326181347328
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/569083142710/locations/us-central1/customJobs/8897818326181347328')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/8897818326181347328?project=569083142710
CustomJob projects/569083142710/locations/us-central1/customJobs/8897818326181347328 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/569083142710/locations/us-central1/customJobs/8897818326181347328 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/569083142710/locations/us-central1/customJobs/8897818326181347328 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/569083142710/locations/us-central1/customJobs/8897818326181347328 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/569083142710/locations/us-central1/customJobs/8897818

## Upload Model to Model Registry

In [11]:
model = aip.Model.upload(
    display_name='sklearn-ecommerce-1',
    artifact_uri=f'{MODEL_URI}/model',
    serving_container_image_uri=IMAGE_PREDICTION_URI,
    serving_container_predict_route='/predict',
    serving_container_health_route='/health'
)

/home/sockcop/.local/lib/python3.9/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Creating Model
Create Model backing LRO: projects/569083142710/locations/us-central1/models/2076214403799187456/operations/5897972579766894592


/home/sockcop/.local/lib/python3.9/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/sockcop/.local/lib/python3.9/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accoun

Model created. Resource name: projects/569083142710/locations/us-central1/models/2076214403799187456@1
To use this Model in another session:
model = aiplatform.Model('projects/569083142710/locations/us-central1/models/2076214403799187456@1')


## Deploy Model for Online Prediction (Vertex Endpoints)

In [12]:
endpoint = model.deploy(
    deployed_model_display_name='sklearn-ecommerce',
    machine_type='n1-standard-2',
    min_replica_count=1,
    max_replica_count=1
)

Creating Endpoint
Create Endpoint backing LRO: projects/569083142710/locations/us-central1/endpoints/8417214509415923712/operations/7127455278039040000
Endpoint created. Resource name: projects/569083142710/locations/us-central1/endpoints/8417214509415923712
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/569083142710/locations/us-central1/endpoints/8417214509415923712')
Deploying model to Endpoint : projects/569083142710/locations/us-central1/endpoints/8417214509415923712
Deploy Endpoint model backing LRO: projects/569083142710/locations/us-central1/endpoints/8417214509415923712/operations/6355087941944999936
Endpoint model deployed. Resource name: projects/569083142710/locations/us-central1/endpoints/8417214509415923712


## Testing (Predictions)

In [13]:
import json

from google.api import httpbody_pb2
from google.cloud import aiplatform_v1

DATA = {
    "signature_name": "predict",
    "instances": [
        {
	        "latest_ecommerce_progress": 0,
            "bounces": 1,
            "time_on_site": 0,
            "pageviews": 1,
            "source": "google",
            "medium": "organic",
            "channel_grouping": "Organic Search",
            "device_category": "desktop",
            "country": "India"
        }
    ],
}

http_body = httpbody_pb2.HttpBody(
    data=json.dumps(DATA).encode("utf-8"),
    content_type="application/json",
)

req = aiplatform_v1.RawPredictRequest(
    http_body=http_body, endpoint=endpoint.resource_name
)

In [15]:

API_ENDPOINT = "{}-aiplatform.googleapis.com".format('us-central1')
client_options = {"api_endpoint": API_ENDPOINT}

pred_client = aip.gapic.PredictionServiceClient(client_options=client_options)

response = pred_client.raw_predict(req)
print(response)

/home/sockcop/.local/lib/python3.9/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


content_type: "application/json"
data: "{\"predictions\":[[0]]}"



## Delete and Clean Components

In [17]:
endpoint.undeploy_all()
model.delete

Undeploying Endpoint model: projects/569083142710/locations/us-central1/endpoints/8417214509415923712
Undeploy Endpoint model backing LRO: projects/569083142710/locations/us-central1/endpoints/8417214509415923712/operations/5365421923830333440
Endpoint model undeployed. Resource name: projects/569083142710/locations/us-central1/endpoints/8417214509415923712


<bound method VertexAiResourceNounWithFutureManager.delete of <google.cloud.aiplatform.models.Model object at 0x7a98a9fdeac0> 
resource name: projects/569083142710/locations/us-central1/models/2076214403799187456>